In [20]:
from typing import Literal
from typing import Annotated, Optional, Dict, List
from typing_extensions import TypedDict
import ollama
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState, END
from langgraph.types import Command
import subprocess
import base64
from pathlib import Path

In [22]:
image_path = Path("images") / "sample_image.jpeg"
with open(image_path, "rb") as img_file:
    image_data = base64.b64encode(img_file.read()).decode("utf-8")

In [26]:
image_data

'...'

In [27]:
from pathlib import Path

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    



response = ollama.chat(
    model='llama3.2-vision',
    messages = [{
    'role': 'user', 
    'content': 'what is this image about?',
    'images': [encode_image(Path("images")/"sample_image.jpeg")]  # Use encoded data
}]
)




In [28]:
response['message']

Message(role='assistant', content="This image appears to be a nutrition label for Cheerios cereal, providing information on the product's ingredients and nutritional content.", images=None, tool_calls=None)

In [16]:
from pathlib import Path
response = ollama.chat(
        model='llama3.2-vision',
        messages=[
            {'role': 'system', 'content': 'Extract nutrition facts as JSON. Return only valid JSON.'},
            {'role': 'user', 'content': 'Process this food label:', 'images': [Path("images") / "sample_image.jpeg"]}
        ]
    )



print(response)

model='llama3.2-vision' created_at='2025-05-03T17:47:02.605182Z' done=True done_reason='stop' total_duration=43574412542 load_duration=1188685375 prompt_eval_count=33 prompt_eval_duration=36134994458 eval_count=148 eval_duration=6200746458 message=Message(role='assistant', content='Here is the processed nutrition information in JSON format:\n\n{"serving_size": 1, "servings_per_container": 60, "calories": 120, "total_fat": 2g, "saturated_fat": 0.5g, "trans_fat": 0g, "cholesterol": 0mg, "sodium": 110mg, "total_carbohydrates": 24g, "dietary_fiber": 1g, "sugars": 8g, "protein": 2g, "vitamin_a": 10%, "vitamin_c": 4%, "calcium": 20%, "iron": 6%}', images=None, tool_calls=None)


In [34]:
from PIL import Image
import base64
from io import BytesIO
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage

def convert_to_base64(pil_image):
    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


pil_image = Image.open("images/sample_image.jpeg")
image_b64 = convert_to_base64(pil_image)


content = [
    {"type": "image_url", "image_url": f"data:image/jpeg;base64,{image_b64}"},
    {"type": "text", "text": "Process this food label"}
]

llm = ChatOllama(model="llama3.2-vision", temperature = 0.7)
response = llm.invoke([SystemMessage(content="Extract nutrition facts as JSON. Return only valid JSON."),HumanMessage(content=content)])
print(response.content)

Here is the nutrition information in a clear and structured format:

**Nutrition Facts**

* **Serving Size**: 1 cup (28g)
* **Servings Per Container**: 6
* **Amount Per Serving**: 28g
* **Calories**: 130
* **Total Fat**: 2g
	+ Saturated Fat: 0.5g
	+ Trans Fat: 0g
* **Cholesterol**: 10mg
* **Sodium**: 150mg
* **Total Carbohydrates**: 27g
	+ Dietary Fiber: 1g
	+ Sugars: 9g
* **Protein**: 2g

**% Daily Value**

* **Vitamin A**: 0%
* **Vitamin C**: 4%
* **Calcium**: 6%
* **Iron**: 10%

**Other Ingredients**

* Whole grain oats
* Modified corn starch
* Sugar
* Salt
* Trisodium phosphate
* Calcium carbonate
* Monoglycerides
* Tocopherols
* Wheat starch
* Annatto
* Vitamins (Niacinamide, Calcium pantothenate, Pyridoxine hydrochloride)
* Minerals (Vitamin B6, Folate)

Note: The "Contains wheat ingredients" statement is not a nutrition fact, but rather an allergen warning.


In [8]:
response['message']

Message(role='assistant', content='Here is the data in a JSON format:\n\n[{\n    "Nutrition Facts": {\n        "Serving Size": 1 cup (28g),\n        "Servings Per Container": 8,\n        "Amount Per Serving": {\n            "Calories": 120,\n            "Total Fat": 2g,\n            "Saturated Fat": 0.5g,\n            "Trans Fat": 0g,\n            "Cholesterol": 0mg,\n            "Sodium": 90mg,\n            "Total Carbohydrates": 25g,\n            "Dietary Fiber": 1g,\n            "Sugars": 7g,\n            "Protein": 2g\n        },\n        "Percent Daily Values are based on a 2000 calorie diet."\n    }\n}]', images=None, tool_calls=None)

In [10]:
label_data = response['message']

In [12]:
question = "Does the item have sugar in it? if yes how much per serving?"
llm = ChatOllama(
    model="llama3.2",  
    temperature=0.7  
)

messages = [
        SystemMessage(content='Extract nutrition facts as JSON. Return only valid JSON.'),
        HumanMessage(content=f'Please help answer the user question using the below food labels I extracted from the image provided by the user. user_question: {question} /n label_data = {label_data}')
    ]


print(llm.invoke(messages).content)

Based on the provided label data, here is the extracted information as JSON:

```json
{
  "has_sugar": true,
  "sugar_per_serving": {
    "amount": 7g,
    "unit": "g"
  }
}
```

This JSON object indicates that the item contains sugar and provides the amount of sugar per serving, which is 7 grams.
